This notebook makes use of https://github.com/akabe/ocaml-jupyter with ocaml 4.02.3

# Prime fields
Let's use our module to build Frobenius fields, aka prime fields $ \mathbf{F}_p $.

First, we need to load the functor that makes a field based on a characteristic `carac`, which must be a prime.

In [24]:
#use "./src/frobenius.ml"

module type FrobT =
  sig
    type t
    val zero : t
    val un : t
    val alpha : t
    val carac : int
    val dim : int
    val cardinal : int
    val to_int : t -> int
    val of_int : int -> t
    val to_list : t -> int list
    val elt_of_list_int : int list -> t
    val print : t -> unit
    val print_table : unit -> unit
    val add : t -> t -> t
    val sub : t -> t -> t
    val opp : t -> t
    val mult : t -> t -> t
    val inv : t -> t
    val div : t -> t -> t
    val pow : t -> int -> t
    val random : unit -> t
  end
module Frobenius : functor (Taille : sig val carac : int end) -> FrobT


Now, let's build $ \mathbf{F}_3  = \mathbf{Z}/(3\mathbf{Z})$ :

In [25]:
module T3 = struct
    let carac = 3
end

module F3 = Frobenius(T3)

module T3 : sig val carac : int end


module F3 :
  sig
    type t = Frobenius(T3).t
    val zero : t
    val un : t
    val alpha : t
    val carac : int
    val dim : int
    val cardinal : int
    val to_int : t -> int
    val of_int : int -> t
    val to_list : t -> int list
    val elt_of_list_int : int list -> t
    val print : t -> unit
    val print_table : unit -> unit
    val add : t -> t -> t
    val sub : t -> t -> t
    val opp : t -> t
    val mult : t -> t -> t
    val inv : t -> t
    val div : t -> t -> t
    val pow : t -> int -> t
    val random : unit -> t
  end


In [26]:
F3.(print alpha)

- : unit = ()


Unfortunately, we cannot print in here directly. We'll just need to flush standard output, which can be done easily  done `print_newline()` or `print_endline()` cf https://caml.inria.fr/pub/docs/manual-ocaml/libref/Pervasives.html :

In [27]:
print_newline ()

- : unit = ()


2%3


We can now see that the 2 is a (the) primitive element in this field. 
If we need more flexible representation, we can just use other primitives :

In [28]:
F3.(to_int alpha) |> print_int ; print_newline ()

- : unit = ()


2


Let's extend this field to make $ \mathbf{F}_9 $. To do this, we're actually going to build $ \mathbf{F}_3[X]/(f(x))$ where $f(x)$ is an irreducible polynomial of degree 2.
First, we need to make polynoms out of these. Then, the module Cantorzass factorizes a cyclotomic polynomial and exten

In [29]:
#mod_use "./src/polynome.ml"
#mod_use "./src/cantorzass.ml"
#mod_use "./src/cyclo.ml"
#use "./src/extensions.ml"

module Polynome :
  sig
    module type CorpsType =
      sig
        type t
        val carac : int
        val cardinal : int
        val dim : int
        val zero : t
        val un : t
        val random : unit -> t
        val print : t -> unit
        val opp : t -> t
        val inv : t -> t
        val add : t -> t -> t
        val mult : t -> t -> t
        val sub : t -> t -> t
        val div : t -> t -> t
        val pow : t -> int -> t
      end
    module type PolyFunctorType =
      functor (Corps : CorpsType) ->
        sig
          type elt = Corps.t
          type t
          val carac : int
          val cardinal : int
          val poly_of_list : elt list -> t
          val list_of_poly : t -> elt list
          val nul : t
          val un : t
          val coef_dom : t -> elt
          val map : (elt -> elt) -> t -> t
          val evaluer : t -> elt -> elt
          val normalise : t -> t
          val unitaire : t -> t
          val decale : t -> int -> t
    

In [30]:
module T6 = struct
    let carac = 3
    let dim = 2
end

module F9 = ExtensionOpt(F3)(T6)

module T6 : sig val carac : int val dim : int end


module F9 :
  sig
    type t = ExtensionOpt(F3)(T6).t
    exception ExtraireImpossible
    val carac : int
    val dim : int
    val cardinal : int
    val zero : t
    val of_int : int -> t
    val to_int : t -> int
    val to_list : t -> int list
    val elt_of_list_int : int list -> t
    val extraire : t -> F3.t
    val incorporer : F3.t -> t
    val un : t
    val alpha : t
    val add : t -> t -> t
    val sub : t -> t -> t
    val opp : t -> t
    val mult : t -> t -> t
    val inv : t -> t
    val div : t -> t -> t
    val pow : t -> int -> t
    val print : t -> unit
    val print_table : unit -> unit
    val random : unit -> t
  end


In [31]:
let _ = 
    let open F9 in
    let rec enum x = match x with
        | a when a=un -> print un ; print_newline() ;
        | a -> print a ; print_newline() ; enum (mult a alpha)
    in enum alpha
;
print_newline() ; F9.print_table ()

a^1
a^2
a^3
a^4
a^5
a^6
a^7
a^0

1%3X^0

0%3X^0 + 1%3X^1

1%3X^0 + 1%3X^1

1%3X^0 + 2%3X^1

2%3X^0

0%3X^0 + 2%3X^1

2%3X^0 + 2%3X^1

2%3X^0 + 1%3X^1



- : unit = ()
